## Vector Stores and Retrievers

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
groqApiKey = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
from langchain_groq     import ChatGroq

LLM = ChatGroq(groq_api_key = groqApiKey, model = "Llama3-8b-8192")
LLM

In [ ]:
from langchain_huggingface      import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
documents

In [ ]:
from langchain_chroma       import Chroma

vectorStore = Chroma.from_documents(documents, embedding = embeddings)
vectorStore

In [ ]:
vectorStore.similarity_search("cat")

In [ ]:
vectorStore.similarity_search_with_score("cat")

## Async Query

In [ ]:
await vectorStore.asimilarity_search("cat")